<a href="https://colab.research.google.com/github/ktam069/Google_Colab_Audiovisual/blob/master/AV_Layers_on_CPU_Full_No_BLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connect to Google Drive to access npy dataset of audio input for training/testing

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Main program - Part IV Project 80 Basic Version

# import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa
from scipy.io import wavfile

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Input, Dense, Conv2D, LSTM, Bidirectional
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten, TimeDistributed, Reshape
from tensorflow.keras.callbacks import ModelCheckpoint

In [0]:
import os
# import sys
import glob
from datetime import datetime

In [5]:
# Main program - Part IV Project 80 Basic Version

# import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa
from scipy.io import wavfile

import tensorflow as tf
from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dense, Conv2D, LSTM, Bidirectional
from keras.layers import BatchNormalization, Activation, Flatten, TimeDistributed, Reshape
from keras.callbacks import ModelCheckpoint
from keras.layers import concatenate
from keras.layers import Lambda

import os
# import sys
import glob
from datetime import datetime

# ===== Settings =====

# Display options
PRINT_DATA = False
DISPLAY_GRAPHS = False

# Data transformation/processing
NORMALISE_DATA = False
POWER_ENCODE = True

# Use float32 format for dataset ndarrays
USE_FLOAT32 = True

# Range of data to use for training (excludes END_ID)
START_ID = 0
END_ID = 21

# Sampling rate
SAMPLING_RATE = 16000

# Filepaths to the locations for input audio-visual files
# path_to_data = "./data/"
path_to_data_audio = "./data/audio/audio_train/"
path_to_data_video = "./data/video/face_input/"
# audio_data_name = "trim_audio_train%d.wav"

# Filepaths to the locations for saved data and models
path_to_models = "./saved_models/"
path_to_saved_datasets = "./dataset_npy/"
path_to_outputs = "./output_wavs/"

# ====================

'''def convolution_model(num_speakers=2):'''

num_speakers=2

# == Audio convolution layers ==

# Input layer for functional model
audio_inputs = Input(shape=(298, 257, 2))

# Convolution layers
model = Conv2D(96, kernel_size=(1,7), padding='same', dilation_rate=(1,1))(audio_inputs)
model = BatchNormalization()(model)
model = Activation("relu")(model)

model = Conv2D(96, kernel_size=(7,1), padding='same', dilation_rate=(1,1))(model)
model = BatchNormalization()(model)
model = Activation("relu")(model)

model = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(1,1))(model)
model = BatchNormalization()(model)
model = Activation("relu")(model)

model = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(2,1))(model)
model = BatchNormalization()(model)
model = Activation("relu")(model)

model = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(4,1))(model)
model = BatchNormalization()(model)
model = Activation("relu")(model)

model = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(8,1))(model)
model = BatchNormalization()(model)
model = Activation("relu")(model)

model = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(16,1))(model)
model = BatchNormalization()(model)
model = Activation("relu")(model)

model = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(32,1))(model)
model = BatchNormalization()(model)
model = Activation("relu")(model)

model = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(1,1))(model)
model = BatchNormalization()(model)
model = Activation("relu")(model)

model = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(2,2))(model)
model = BatchNormalization()(model)
model = Activation("relu")(model)

model = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(4,4))(model)
model = BatchNormalization()(model)
model = Activation("relu")(model)

model = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(8,8))(model)
model = BatchNormalization()(model)
model = Activation("relu")(model)

model = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(16,16))(model)
model = BatchNormalization()(model)
model = Activation("relu")(model)

model = Conv2D(96, kernel_size=(5,5), padding='same', dilation_rate=(32,32))(model)
model = BatchNormalization()(model)
model = Activation("relu")(model)

model = Conv2D(8, kernel_size=(1,1), padding='same', dilation_rate=(1,1))(model)
model = BatchNormalization()(model)
model = Activation("relu")(model)

audio_outputs = Reshape((298, 8*257))(model)

audio_model = Model(audio_inputs, audio_outputs)

# == Visual convolution layers ==

visual_inputs_list = []
visual_outputs_list = []

for i in range(num_speakers):
  # Input layer for functional model
  # TODO: check the correctness of the input layer
  visual_inputs = Input(shape=(75, 1, 1024))

  # Convolution layers
  model = Conv2D(256, kernel_size=(7,1), padding='same', dilation_rate=(1,1))(visual_inputs)
  model = BatchNormalization()(model)
  model = Activation("relu")(model)

  model = Conv2D(256, kernel_size=(5,1), padding='same', dilation_rate=(1,1))(model)
  model = BatchNormalization()(model)
  model = Activation("relu")(model)

  model = Conv2D(256, kernel_size=(5,1), padding='same', dilation_rate=(2,1))(model)
  model = BatchNormalization()(model)
  model = Activation("relu")(model)

  model = Conv2D(256, kernel_size=(5,1), padding='same', dilation_rate=(4,1))(model)
  model = BatchNormalization()(model)
  model = Activation("relu")(model)

  model = Conv2D(256, kernel_size=(5,1), padding='same', dilation_rate=(8,1))(model)
  model = BatchNormalization()(model)
  model = Activation("relu")(model)

  model = Conv2D(256, kernel_size=(5,1), padding='same', dilation_rate=(16,1))(model)
  model = BatchNormalization()(model)
  model = Activation("relu")(model)

  model = Reshape((75, 256, 1))(model)

  # TODO - upsampling code taken from other repo
  def UpSampling2DBilinear(size):
    return Lambda(lambda x: tf.image.resize_bilinear(x, size, align_corners=True))

  model = UpSampling2DBilinear((298, 256))(model)

  visual_outputs = Reshape((298, 256))(model)

  visual_model = Model(visual_inputs, visual_outputs)

  # visual_model.summary()

  visual_inputs_list.append(visual_inputs)
  visual_outputs_list.append(visual_outputs)

# == AV fused neural network ==

fused_model = concatenate([audio_outputs]+visual_outputs_list, axis=2)

# AV fusion step(s)
fused_model = TimeDistributed(Flatten())(fused_model)

# BLSTM
new_matrix_length = 400
# fused_model = Bidirectional(LSTM(new_matrix_length//2, return_sequences=True, input_shape=(298, 257*8)))(fused_model)
#fused_model = Bidirectional(LSTM(new_matrix_length//2, return_sequences=True, input_shape=(298, 257*8 + 256*num_speakers)))(fused_model)

# Fully connected layers
fused_model = Dense(600, activation="relu")(fused_model)
fused_model = Dense(600, activation="relu")(fused_model)
fused_model = Dense(600, activation="relu")(fused_model)

# Output layer (i.e. complex masks)
fused_model = Dense(257*2*num_speakers, activation="sigmoid")(fused_model)				# TODO: check if this is more correct (based on the paper)
outputs_complex_masks = Reshape((298, 257, 2, num_speakers), name="output_layer")(fused_model)

av_model = Model(inputs=[audio_inputs]+visual_inputs_list, outputs=outputs_complex_masks)

model = av_model

# Print the output shapes of each model layer
for layer in model.layers:
  name = layer.get_config()["name"]
  if "batch_normal" in name or "activation" in name:
    continue
  print(layer.output_shape, "\t", name)

# Alternatively, print the default keras model summary
model.summary()

# Compile the model before training
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])


Using TensorFlow backend.



(None, 298, 257, 2) 	 input_1
(None, 298, 257, 96) 	 conv2d_1
(None, 298, 257, 96) 	 conv2d_2
(None, 298, 257, 96) 	 conv2d_3
(None, 298, 257, 96) 	 conv2d_4
(None, 298, 257, 96) 	 conv2d_5
(None, 298, 257, 96) 	 conv2d_6
(None, 298, 257, 96) 	 conv2d_7
(None, 298, 257, 96) 	 conv2d_8
(None, 298, 257, 96) 	 conv2d_9
(None, 75, 1, 1024) 	 input_2
(None, 75, 1, 1024) 	 input_3
(None, 75, 1, 256) 	 conv2d_16
(None, 75, 1, 256) 	 conv2d_22
(None, 298, 257, 96) 	 conv2d_10
(None, 75, 1, 256) 	 conv2d_17
(None, 75, 1, 256) 	 conv2d_23
(None, 298, 257, 96) 	 conv2d_11
(None, 75, 1, 256) 	 conv2d_18
(None, 75, 1, 256) 	 conv2d_24
(None, 298, 257, 96) 	 conv2d_12
(None, 75, 1, 256) 	 conv2d_19
(None, 75, 1, 256) 	 conv2d_25
(None, 298, 257, 96) 	 conv2d_13
(None, 75, 1, 256) 	 conv2d_20
(None, 75, 1, 256) 	 conv2d_26
(None, 298, 257, 96) 	 conv2d_14
(None, 75, 1, 256) 	 conv2d_21
(None, 75, 1, 256) 	 conv2d_27
(None, 298, 257, 8) 	 conv2d_15
(None, 75, 256, 1) 	 reshape_2
(None, 75, 256, 1) 	 

In [0]:
!cp '/content/gdrive/My Drive/Part IV Project/Colab Notebooks/dataset_npy/dataset_train_0-20.npy' "dataset_train_0-20.npy"

In [7]:
!ls

dataset_train_0-20.npy	gdrive	sample_data


In [0]:
dataset_train = np.load("dataset_train_0-20.npy")

In [0]:
# Split dataset into inputs and ground truths
x_train = dataset_train[:,:,:,:,0 ]
y_train = dataset_train[:,:,:,:,1:]


x_visual = np.random.rand(len(x_train), 75, 1, 1024)

In [0]:
import time

In [12]:
times = []
num_tests = 10

for i in range(num_tests):
	'''Predict data'''
	start_time = time.time()
	#model.predict(x_train[i:i+1])
	model.predict([x_train[i:i+1], x_visual[i:i+1], x_visual[i:i+1]])
	end_time = time.time()
	runtime = end_time-start_time
	times.append(runtime)
	print("Time taken for run %d:"%(i+1), runtime)

avg = sum(times) / len(times)
print("Number of test:", len(times))
print("Test times:", times)
print("Average running time:", avg)

Time taken for run 1: 6.289369583129883
Time taken for run 2: 6.307868957519531
Time taken for run 3: 6.301701784133911
Time taken for run 4: 6.303648471832275
Time taken for run 5: 6.276608228683472
Time taken for run 6: 6.290634632110596
Time taken for run 7: 6.262272119522095
Time taken for run 8: 6.30098819732666
Time taken for run 9: 6.272015571594238
Time taken for run 10: 6.272173643112183
Number of test: 10
Test times: [6.289369583129883, 6.307868957519531, 6.301701784133911, 6.303648471832275, 6.276608228683472, 6.290634632110596, 6.262272119522095, 6.30098819732666, 6.272015571594238, 6.272173643112183]
Average running time: 6.287728118896484


In [0]:
fname = "ablation_basic_times_full_noBLSTM.csv"
np.savetxt(fname, times, delimiter=",")

In [0]:
new_dir = "/content/gdrive/My Drive/Part IV Project/Colab Testing Structure/AV_basic_ablation_output/"
if not os.path.isdir(new_dir):
  os.mkdir(new_dir)

In [0]:
!cp $fname "{new_dir}{fname}"

In [16]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 298, 257, 2)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 298, 257, 96) 1440        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 298, 257, 96) 384         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 298, 257, 96) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [17]:
audio_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 298, 257, 2)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 298, 257, 96)      1440      
_________________________________________________________________
batch_normalization_1 (Batch (None, 298, 257, 96)      384       
_________________________________________________________________
activation_1 (Activation)    (None, 298, 257, 96)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 298, 257, 96)      64608     
_________________________________________________________________
batch_normalization_2 (Batch (None, 298, 257, 96)      384       
_________________________________________________________________
activation_2 (Activation)    (None, 298, 257, 96)      0   

In [18]:
visual_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 75, 1, 1024)       0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 75, 1, 256)        1835264   
_________________________________________________________________
batch_normalization_22 (Batc (None, 75, 1, 256)        1024      
_________________________________________________________________
activation_22 (Activation)   (None, 75, 1, 256)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 75, 1, 256)        327936    
_________________________________________________________________
batch_normalization_23 (Batc (None, 75, 1, 256)        1024      
_________________________________________________________________
activation_23 (Activation)   (None, 75, 1, 256)        0   